In [1]:
%load_ext autoreload
%autoreload 2
from constants import *
from evaluator import *
from model import *
from mutator import *
from prompt import *
from task import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
import re

def extract_last_numeric_value(input_string):
    # Regular expression pattern to extract all numeric values with periods and ignoring commas
    pattern = r'[\d,.]+'

    # Find all matches using re.finditer()
    matches = re.finditer(pattern, input_string)

    # Initialize a variable to store the last numeric value
    last_numeric_value = None

    # Iterate through the matches and update the last_numeric_value
    for match in matches:
        numeric_value = match.group()
        # Remove commas if needed
        numeric_value = numeric_value.replace(",", "")
        last_numeric_value = numeric_value

    return last_numeric_value

In [3]:
def evaluate_func(orig, pred):
    orig_value = extract_last_numeric_value(orig)
    pred_value = extract_last_numeric_value(pred)
    try:
        return abs(float(orig_value) - float(pred_value)) < 1e-6
    except:
        return False

In [4]:
from datasets import load_dataset

# mutation_model = Model(provider="huggingface", model_name="mistralai/Mistral-7B-Instruct-v0.1")
mutation_model = Model(provider="quantized_llama", model_name="TheBloke/Mistral-7B-OpenOrca-GGUF", model_file="mistral-7b-openorca.Q4_K_M.gguf", model_type="mistral")
evaluation_model = mutation_model

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
system_instruction = ""
thinking_style = ""
task = Task(load_dataset('gsm8k', 'main'),
            'Solve the math word problem, giving your answer as an arabic numeral.',
            evaluate_func)
initial_prompt = Prompt(task.initial_prompt, system_instruction, thinking_style, None)

In [6]:
task.test()

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})

In [ ]:
evaluator = Evaluator()
all_pop = [initial_prompt]
scores = evaluator.evaluate(evaluation_model, task, "one", all_pop, len(task.test()))

# Print prompt and their score
prompt_scores = [(all_pop[i].get_accuracy(), all_pop[i]) for i in range(len(all_pop))]
prompt_scores = sorted(prompt_scores, key=lambda x: x[0], reverse=True)
print("Score  | Evals | Prompt")
for _, prompt in prompt_scores:
    print(f"{prompt.get_accuracy() * 100:5.2f}% | {prompt.get_num_evals():5d} | {prompt}")

# Survival of the fittest
# TODO: Create a more elaborate selection mechanism that keeps some weaker member
pop = [prompt_scores[i][1] for i in range(NUM_POPULATION)]

# Record time
end_time = time.time()
execution_time = end_time - start_time
print("Execution time: {:.2f} seconds".format(execution_time))

Evaluating test samples: [537, 658, 630, 1242, 927, 1001, 917, 280, 302, 476, 269, 104, 1182, 506, 574, 161, 1093, 1036, 26, 1233, 496, 1011, 366, 880, 1223, 1305, 508, 890, 1015, 697, 88, 1200, 510, 585, 62, 824, 109, 1092, 263, 282, 274, 1136, 716, 381, 324, 918, 409, 1309, 1039, 742, 544, 1003, 855, 724, 1229, 826, 349, 446, 203, 339, 994, 870, 1152, 522, 1288, 55, 1038, 597, 59, 1109, 1155, 19, 983, 941, 536, 1173, 125, 398, 1141, 1232, 578, 395, 313, 812, 134, 94, 886, 156, 111, 707, 1076, 1208, 1075, 143, 893, 520, 889, 881, 1284, 1116, 238, 1002, 241, 769, 1165, 166, 504, 215, 713, 461, 1197, 1135, 1158, 364, 691, 1040, 348, 1150, 861, 296, 986, 689, 160, 694, 250, 525, 612, 1168, 539, 376, 494, 1026, 844, 464, 1035, 1072, 968, 717, 218, 347, 455, 157, 518, 458, 764, 725, 833, 590, 484, 312, 860, 1196, 645, 28, 332, 859, 98, 1180, 427, 1139, 1059, 761, 1273, 699, 32, 825, 907, 767, 1004, 320, 516, 44, 204, 710, 445, 671, 1227, 571, 679, 674, 198, 921, 781, 448, 829, 459, 617, 66